In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
        .master("local[*]")
    .appName('spark_sql')
    .getOrCreate()
)

# configurando as opções e lendo o csv
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true') # tenta inferir o schema
    .option('enconding', 'ISO-8859-1') # as vezes está especificado no arquivo
    .csv('./precos-gasolina-etanol-10.csv') # acesso relativo ao arquivo
)

df.printSchema()

your 131072x1 screen size is bogus. expect trouble
24/08/16 14:04:54 WARN Utils: Your hostname, PGONGARTI11 resolves to a loopback address: 127.0.1.1; using 172.30.40.69 instead (on interface eth0)
24/08/16 14:04:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/16 14:05:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [2]:
df.createOrReplaceTempView('combustiveis') # view temporária

In [3]:
spark.sql("""
    select * from combustiveis where `Valor de Compra` is not null
""").show()

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [4]:
view_precos = spark.sql("""
    select
        `Estado - Sigla`,
        `Produto`,
        regexp_replace(`Valor de Venda`, ',', '.') as `Valor de Venda`,
        `Unidade de Medida`
    from combustiveis
""")

In [5]:
view_precos.createOrReplaceTempView('view_precos')
view_precos.show()

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AL|          GASOLINA|          6.08|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.08|       R$ / litro|
|            AL|            ETANOL|          4.78|       R$ / litro|
|            AL|          GASOLINA|          5.79|       R$ / litro|
|            AL|            ETANOL|          4.29|       R$ / litro|
|            AL|          GASOLINA|          5.89|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.09|       R$ / litro|
|            AL|            ETANOL|          4.49|       R$ / litro|
|            AL|          GASOLINA|          5.89|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.09|       R$ / litro|
|            AL|            ETANOL|          4.99|       R$ / litro|
|            AL|          GASOLINA

In [9]:
spark.sql("""
    select
        `Estado - Sigla`,
        `Produto`,
        `Unidade de Medida`,
        min(`Valor de Venda`) as menor_valor,
        max(`Valor de Venda`) as maior_valor,
        max(`Valor de Venda`) - min(`Valor de Venda`) as diferenca
    from view_precos
    group by all
    order by diferenca DESC
""").show()

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       4.79|       8.69|3.8999999999999995|
|            SP|            ETANOL|       R$ / litro|       2.78|       6.19|3.4100000000000006|
|            SP|          GASOLINA|       R$ / litro|       4.59|       7.59|               3.0|
|            PA|            ETANOL|       R$ / litro|       3.85|        6.6|2.7499999999999996|
|            RS|            ETANOL|       R$ / litro|       3.88|       6.29|              2.41|
|            BA|          GASOLINA|       R$ / litro|       4.69|       6.98|              2.29|
|            SC|            ETANOL|       R$ / litro|       3.84|       5.89|              2.05|
|            AL|            ET